In [1]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [6]:
import psycopg2

# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\server-ca.pem"
}

try:
    # Establishing the connection
    post_conn = psycopg2.connect(**db_params)
    
    # Creating a cursor object using the connection
    post_cur = post_conn.cursor()
    
    # Test the connection by executing a simple query
    post_cur.execute('''
select
	poh.po_header_id,
	pol.pol_line_id ,
	poh.po_num,
	poh.po_date_ordered,
	poh.act_ship_to_fac,
    pol.POL_VENDOR_TI ,
    im.VENDOR_TIE as VENDOR_TI,
    CASE 
        WHEN pol.POL_VENDOR_TI = im.VENDOR_TIE THEN 'Y'
        ELSE 'N'
    END AS VENDOR_TI_CHECK,
    pol.POL_VENDOR_HI,
    im.VENDOR_TIER as VENDOR_HI,
    CASE 
        WHEN pol.POL_VENDOR_HI = im.VENDOR_TIER THEN 'Y'
        ELSE 'N'
    END AS VENDOR_HI_CHECK,
    pol.POL_WHSE_TI,
    im.WHSE_TIE as WHSE_TI,
    CASE 
        WHEN pol.POL_WHSE_TI = im.WHSE_TIE THEN 'Y'
        ELSE 'N'
    END AS WHSE_TI_CHECK,
    pol.POL_WHSE_HI,
    im.WHSE_TIER as WHSE_HI,
    CASE 
        WHEN pol.POL_WHSE_HI = im.WHSE_TIER THEN 'Y'
        ELSE 'N'
    END AS WHSE_HI_CHECK
    ,
    pol.ssic,
    pol.item_warehouse_code,
    im.item_ssic,
    poh.po_status,
    poh.received_date
--    pol.*,
--    poh.*,
FROM 
    XXPO_PO_HEADERS poh
JOIN 
    XXPO_PO_LINES pol
    ON pol.po_header_id = poh.po_header_id
JOIN 
    XXPO_ITEM_MASTER im
    ON LPAD(pol.ssic, 10, '0') = im.item_ssic
WHERE 
    poh.RECEIVED_DATE IS NOT NULL
    AND poh.PO_STATUS IN ('Open', 'Pre-Approved');
''')
    df = post_cur.fetchall()  # Output should be (1,)
    print(df)
except Exception as e:
    print(f"Error: {e}")

# finally:
#     # Closing the cursor and connection
#     if post_cur:
#         post_cur.close()
#     if post_conn:
#         post_conn.close()


[(102, 512, '132404', None, '303', Decimal('13'), Decimal('13'), 'Y', Decimal('6'), Decimal('6'), 'Y', Decimal('13'), Decimal('13'), 'Y', Decimal('9'), Decimal('9'), 'Y', '20288', '10', '0000020288', 'Open', datetime.date(2024, 8, 20)), (102, 524, '132404', None, '303', Decimal('22'), Decimal('22'), 'Y', Decimal('5'), Decimal('5'), 'Y', Decimal('22'), Decimal('22'), 'Y', Decimal('3'), Decimal('3'), 'Y', '26375', '10', '0000026375', 'Open', datetime.date(2024, 8, 20)), (102, 527, '132404', None, '303', Decimal('10'), Decimal('10'), 'Y', Decimal('7'), Decimal('7'), 'Y', Decimal('10'), Decimal('10'), 'Y', Decimal('7'), Decimal('7'), 'Y', '29771', '10', '0000029771', 'Open', datetime.date(2024, 8, 20)), (102, 530, '132404', None, '303', Decimal('12'), Decimal('12'), 'Y', Decimal('8'), Decimal('8'), 'Y', Decimal('12'), Decimal('12'), 'Y', Decimal('8'), Decimal('8'), 'Y', '29831', '10', '0000029831', 'Open', datetime.date(2024, 8, 20)), (102, 531, '132404', None, '303', Decimal('12'), Decima

In [11]:
try:
    update_query = '''
        WITH records_to_update AS (
            SELECT
                pol.po_header_id,
                pol.pol_line_id,
                pol.pol_po_selection_fac,
                im.VENDOR_TIE,
                im.VENDOR_TIER,
                im.WHSE_TIE,
                im.WHSE_TIER,
                pol.order_qty,
                -- Calculating PALLET_QUANTITY and VEN_PALLET_QTY
                pol.order_qty / (im.VENDOR_TIE * im.VENDOR_TIER) AS PALLET_QUANTITY,
                pol.order_qty / (im.WHSE_TIE * im.WHSE_TIER) AS VEN_PALLET_QTY
            FROM 
                XXPO_PO_HEADERS poh
            JOIN 
                XXPO_PO_LINES pol ON pol.po_header_id = poh.po_header_id
            JOIN 
                XXPO_ITEM_MASTER im ON LPAD(pol.ssic, 10, '0') = im.item_ssic
            WHERE 
                poh.RECEIVED_DATE IS NOT NULL
                AND im.facility_code = pol.pol_po_selection_fac
                AND poh.PO_STATUS IN ('Open', 'Pre-Approved')
                AND (
                    pol.POL_VENDOR_TI <> im.VENDOR_TIE
                    OR pol.POL_VENDOR_HI <> im.VENDOR_TIER
                    OR pol.POL_WHSE_TI <> im.WHSE_TIE
                    OR pol.POL_WHSE_HI <> im.WHSE_TIER
                )
        )
        -- Now update the columns based on the CTE
        UPDATE XXPO_PO_LINES pol
        SET 
            POL_VENDOR_TI = rtu.VENDOR_TIE,
            POL_VENDOR_HI = rtu.VENDOR_TIER,
            POL_WHSE_TI = rtu.WHSE_TIE,
            POL_WHSE_HI = rtu.WHSE_TIER,
            PALLET_QUANTITY = rtu.PALLET_QUANTITY,
            VEN_PALLET_QTY = rtu.VEN_PALLET_QTY
        FROM 
            records_to_update rtu
        WHERE 
            pol.po_header_id = rtu.po_header_id
            AND pol.pol_line_id = rtu.pol_line_id
            AND rtu.pol_po_selection_fac = pol.pol_po_selection_fac;
    '''
    post_cur.execute(update_query)
    cnt = post_cur.rowcount
    print("No of Records updated in XXPO_PO_LINES =", cnt)

except Exception as e:
    print(f"Error while updating the records: {e}")

# # Execute the query using post_cur
# post_cur.execute(query)

# # Commit the transaction to save changes
# post_cur.connection.commit()


In [12]:
update_count = post_cur.rowcount
update_count

4